In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split


heart_disease = fetch_ucirepo(id=45)
X = heart_disease.data.features
Y = heart_disease.data.targets
X.fillna({'ca': X['ca'].median(), 'thal': X['thal'].mode()[0]}, inplace=True)

for index, row in Y.iterrows():
    if row[0] != 0:
        row[0] = 1 #jeżeli nie ma klasy 0 to ma klasę 1

y = Y.to_numpy()
X = (X-X.min())/(X.max()-X.min())
x = X.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2)

x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

print(x_train)

tensor([[0.3750, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5833, 1.0000, 1.0000,  ..., 0.5000, 0.0000, 0.7500],
        [0.8542, 1.0000, 1.0000,  ..., 1.0000, 0.0000, 1.0000],
        ...,
        [0.8542, 1.0000, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        [0.5625, 0.0000, 0.3333,  ..., 0.5000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.3333,  ..., 0.0000, 0.0000, 0.0000]])


c:\users\krzys\desktop\pythonproject\againterpreter\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(x_train.shape[1], 64)
        self.hidden_layer = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.input_layer(x)
        x = self.sigmoid(x)
        x = self.hidden_layer(x)
        x = self.sigmoid(x)
        x = self.output_layer(x)
        return x

model = NeuralNetwork()

In [12]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(x_train, y_train)
custom_batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=custom_batch_size, shuffle=True)

# funkcja kosztu i optimizer
# criterion = nn.BCELoss()  # Binary Cross-Entropy
criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.1)  # Stochastic Gradient Descent optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.008)
optimizer = optim.Adagrad(model.parameters(), lr=0.01)

num_epochs = 10000

for epoch in range(num_epochs):
    for inputs, labels in train_loader:  # iteracja po barchach
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/10000], Loss: 0.3867
Epoch [200/10000], Loss: 0.4173
Epoch [300/10000], Loss: 0.4054
Epoch [400/10000], Loss: 0.3644
Epoch [500/10000], Loss: 0.2761
Epoch [600/10000], Loss: 0.3474
Epoch [700/10000], Loss: 0.3275
Epoch [800/10000], Loss: 0.2460
Epoch [900/10000], Loss: 0.4202
Epoch [1000/10000], Loss: 0.2986
Epoch [1100/10000], Loss: 0.2868
Epoch [1200/10000], Loss: 0.3555
Epoch [1300/10000], Loss: 0.3827
Epoch [1400/10000], Loss: 0.3492
Epoch [1500/10000], Loss: 0.4046
Epoch [1600/10000], Loss: 0.4094
Epoch [1700/10000], Loss: 0.3189
Epoch [1800/10000], Loss: 0.3670
Epoch [1900/10000], Loss: 0.2918
Epoch [2000/10000], Loss: 0.4039
Epoch [2100/10000], Loss: 0.3267
Epoch [2200/10000], Loss: 0.3594
Epoch [2300/10000], Loss: 0.3161
Epoch [2400/10000], Loss: 0.3176
Epoch [2500/10000], Loss: 0.2972
Epoch [2600/10000], Loss: 0.3507
Epoch [2700/10000], Loss: 0.3405
Epoch [2800/10000], Loss: 0.3112
Epoch [2900/10000], Loss: 0.3832
Epoch [3000/10000], Loss: 0.3455
Epoch [3100/10000],

In [13]:
# ewaluacja modelu na zbiorze testowym
from sklearn.metrics import confusion_matrix
with torch.no_grad():
    test_outputs = model(x_test)
    test_outputs = (test_outputs > 0.5).float()
    accuracy = (test_outputs == y_test).sum().item() / len(y_test)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    cm = confusion_matrix(y_test, test_outputs)
    print("Confusion Matrix:")
    print(cm)

Test Accuracy: 81.97%
Confusion Matrix:
[[32  3]
 [ 8 18]]
